In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.
C:\Users\EL FASHKA\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\EL FASHKA\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\EL FASHKA\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\EL FASHKA\anaconda3\lib\site-packages\tensorflow\python\fra

In [2]:
df = pd.read_csv('C:/Users/EL FASHKA/Desktop/FABB/conso.csv', sep = ";", usecols=['Date', 'Heures', 'Consommation', 'Solaire'])
df = df[['Date', 'Heures', 'Consommation', 'Solaire']]
df.head()

,Date,Heures,Consommation,Solaire
0,01/01/2019,00:00,64267,-2
1,01/01/2019,00:30,63209,-2
2,01/01/2019,01:00,60968,-2
3,01/01/2019,01:30,60772,-2
4,01/01/2019,02:00,60164,-2


In [3]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
df.head() 

,Heures,Consommation,Solaire
Date,,,
2019-01-01,00:00,64267,-2
2019-01-01,00:30,63209,-2
2019-01-01,01:00,60968,-2
2019-01-01,01:30,60772,-2
2019-01-01,02:00,60164,-2


In [6]:
train, test = df[:-12], df[-12:]

In [7]:
#data = df.filter(['Consommation'])
#dataset = data.values
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test) 

ValueError: could not convert string to float: '00:00'

In [47]:
n_input = 12
n_features = 1
generator = TimeseriesGenerator(train, train, length = n_input, batch_size = 6 )
model = Sequential()
model.add(LSTM(200, activation = 'relu', input_shape= (n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit_generator(generator, epochs = 1)

Epoch 1/1
5830/5830 [==============================] - 56s 10ms/step - loss: 0.0069


In [54]:
pred_list = []
batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range (n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:, 1:, :], [[pred_list[i]]], axis=1)

In [52]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), index = df[-n_input:].index, columns = ['Predictions'])
df_test = pd.concat([df, df_predict], axis= 1)

ValueError: Shape of passed values is (24, 1), indices imply (12, 1)